In [187]:
import numpy as np
import copy
from random import randint

In [188]:
board = np.asarray([["-"]*8]*8)
print(board.shape)

(8, 8)


In [189]:
class GamePiece:
    def __init__(self, team):
        self.team = team

    def get_team(self):
        return self.team


piece = GamePiece("White")
print("Team:", piece.get_team())  # Output: Team: White


Team: White


In [190]:
class BoardState:
    """State of a Konane Game Board"""

    def __init__(self):
        """Initialize empty array to store the board representation"""
        self.board = np.asarray([[None]*8]*8)

    def initialize_board(self):
        """Populates board with GamePiece objects in the starting configuration"""

        for i in range(len(self.board)):
            for j in range(len(self.board[0])):
                if((i+j)%2 == 0):
                    self.board[i][j] = GamePiece("Black")
                else:
                    self.board[i][j] = GamePiece("White")

    def move_piece(self, srow, scol, erow, ecol):
        """Moves the GamePiece at the starting location to the ending location

        Parameters
        ----------
        srow : int
            Starting row
        scol : int
            Starting column
        erow : int
            Ending row
        ecol : int
            Ending columng

        Returns
        ----------
        board_states : 
            a list of BoardState objects
        """

        if(self.is_valid_position(erow, ecol)):
            temp = copy.deepcopy(self.board[srow][scol])
            self.board[srow][scol] = None
            # vertical move
            if(scol == ecol):
                if(srow < erow):
                    for i in range(srow, erow, 2):
                        self.remove_piece(i+1, scol)
                        # self.board[i+1][scol] = None
                        if(srow > i > erow):
                            print("double move")
                else:
                    for i in range(srow, erow, -2):
                        self.remove_piece(i-1, scol)
                        # self.board[i-1][scol] = None
                        if(erow > i > srow):
                            print("double move")
                        
            # horizontal move
            elif(srow == erow):
                if(scol < ecol):
                    for j in range(scol, ecol, 2):
                        self.remove_piece(srow, j+1)
                        # self.board[srow][j+1] = None
                else:
                    for j in range(scol, ecol, -2):
                        self.remove_piece(srow, j-1)
                        # self.board[srow][j-1] = None
            else:
                print("non orthogonal move")
                return None
        else:
            print("move out of range")
            return None
        
        self.board[erow][ecol] = GamePiece(temp.get_team())
        return(self)

    def remove_piece(self, row, col):
        """Removes the GamePiece object at the specified row and column

        Parameters
        ----------
        row : int
            row where piece is located
        col : inte
            column where piece is located

        Returns
        ----------
        board : 
            the updated BoardState after the removal has been performed
        None :
            None if the remove is invalid
        """
        if(self.is_valid_position(row, col)):
            self.board[row][col] = None
            return self.board
        else:
            print("invalid remove\n")
            return None

    def is_valid_position(self, y, x):
        """Checks if a board position is valid

        Parameters
        ----------
        y : int
            row of position
        x : int
            column of position

        Returns
        ----------
        ret : boolean
            true if position is on the board, false otherwise
        """
        return 0 <= x < 8 and 0 <= y < 8

    def moves_available(self, team):
        """Checks for available moves from the current state of the board

        Parameters
        ----------
        team: str
            "Black" or "White"

        Returns
        ----------
        board_states : list
            list of BoardState objects containing possible moves
        """
        num_moves = 0
        board_states = []
        
        pieces_available = [[],[]]

        for i in range(0,len(self.board)):
            for j in range(0,len(self.board[0])):
                if(self.board[i][j] != None and self.board[i][j].get_team() == team):
                    pieces_available[0].append(i)
                    pieces_available[1].append(j)

        # a = np.where(self.board.get_team() == team)
        for i,j in zip(pieces_available[0],pieces_available[1]):
            # up
            k = i
            while k-2 >= 0:
                bs_cpy = copy.deepcopy(self)
                if(self.board[k-2][j] == None and self.board[k-1][j] != None and self.board[k-1][j].get_team() != team and self.board[k-1][j] != None):
                    num_moves+=1
                    board_states.append(bs_cpy.move_piece(i,j,k-2,j))
                    k-=2
                else:
                    break
            # down
            k = i
            while k+2 < len(self.board):
                bs_cpy = copy.deepcopy(self)
                if(self.board[k+2][j] == None and self.board[k+1][j] != None and self.board[k+1][j].get_team() != team and self.board[k+1][j] != None):
                    num_moves+=1
                    board_states.append(bs_cpy.move_piece(i,j,k+2,j))
                    k+=2
                else:
                    break
            # left
            k = j
            while k-2 >= 0:
                bs_cpy = copy.deepcopy(self)
                if(self.board[i][k-2] == None and self.board[i][k-1] != None and self.board[i][k-1].get_team() != team and self.board[i][k-1] != None):
                    num_moves+=1
                    board_states.append(bs_cpy.move_piece(i,j,i,k-2))
                    k-=2
                else:
                    break
            # right
            k = j
            while k+2 < len(self.board[0]):
                bs_cpy = copy.deepcopy(self)
                if(self.board[i][k+2] == None and self.board[i][k+1] != None and self.board[i][k+1].get_team() != team and self.board[i][k+1] != None):
                    num_moves+=1
                    board_states.append(bs_cpy.move_piece(i,j,i,k+2))
                    k+=2
                else:
                    break
        return board_states

    def print_board(self):
        for i in range(len(board)):
            str = ""
            for j in range(len(board[0])):
                if(self.board[i][j] == None):
                    str = str + "." + " "
                elif(self.board[i][j].get_team() == "Black"):
                    str = str + "x" + " "
                elif(self.board[i][j].get_team() == "White"):
                    str = str + "o" + " "
                else:
                    print("BOARD ERROR")
            print(str)

    

In [191]:
new_game = BoardState()
new_game.initialize_board()
new_game.print_board()

x o x o x o x o 
o x o x o x o x 
x o x o x o x o 
o x o x o x o x 
x o x o x o x o 
o x o x o x o x 
x o x o x o x o 
o x o x o x o x 


In [192]:
new_game.remove_piece(3,3)
new_game.remove_piece(4,3)
new_game.print_board()

x o x o x o x o 
o x o x o x o x 
x o x o x o x o 
o x o . o x o x 
x o x . x o x o 
o x o x o x o x 
x o x o x o x o 
o x o x o x o x 


In [193]:
for brd in new_game.moves_available("Black"):
    brd.print_board()
    print("\n")

x o x o x o x o 
o x o . o x o x 
x o x . x o x o 
o x o x o x o x 
x o x . x o x o 
o x o x o x o x 
x o x o x o x o 
o x o x o x o x 


x o x o x o x o 
o x o x o x o x 
x o x o x o x o 
o . . x o x o x 
x o x . x o x o 
o x o x o x o x 
x o x o x o x o 
o x o x o x o x 


x o x o x o x o 
o x o x o x o x 
x o x o x o x o 
o x o x . . o x 
x o x . x o x o 
o x o x o x o x 
x o x o x o x o 
o x o x o x o x 




In [194]:
def generate_st_tree(board_st, plyr_team, mx_depth):

    q = []
    node = 0
    q.append(node)
    state_tree_dict[node] = board_st
    heuristic = [None]
    parents = [-1]
    depth = [0]
    player_team = plyr_team
    opponent_team = ""
    team = player_team
    max_depth = mx_depth

    if(player_team == "White"):
        opponent_team = "Black"
    elif(player_team == "Black"):
        opponent_team = "White"
    else: 
        print("team assignment failure")
        return None

    while len(q)>0:
        # remove front of queue
        p = q.pop(0)

        if depth[p] < max_depth:
            # find board state corresponding to node
            current_state = state_tree_dict[p]
            # initialize empty board_states
            board_states = 0
            # set team depending on depth
            if(depth[p] % 2 == 0):
                # get all available board_states 
                board_states = current_state.moves_available(player_team)
            else:
                board_states = current_state.moves_available(opponent_team)
            

            for brd in board_states:
                # increment node
                node += 1

                for k in range(len(parents), node+1):
                    parents.append(-1)
                
                for k in range(len(depth), node+1):
                    depth.append(-1)
                
                for k in range(len(heuristic), node+1):
                    heuristic.append(None)

                q.append(node)
                state_tree_dict[node] = brd
                parents[node] = p
                depth[node] = depth[p] + 1
                if(depth[node] % 2 == 0):
                    heuristic[node] = len(brd.moves_available(player_team))-len(brd.moves_available(opponent_team))
                else:
                    heuristic[node] = len(brd.moves_available(opponent_team))-len(brd.moves_available(player_team))
        else:
            break
    
    ret_dict = {}
    ret_dict[0] = state_tree_dict
    ret_dict[1] = heuristic
    ret_dict[2] = depth
    ret_dict[3] = parents
    return(ret_dict)

# for i in state_tree_dict:
#     state_tree_dict[i].print_board()
#     print()


In [195]:
arr = generate_st_tree(new_game, "White", 2)

In [196]:
for i in arr:
    print(arr[i])

{0: <__main__.BoardState object at 0x121a8b5b0>, 1: <__main__.BoardState object at 0x1227e3a30>, 2: <__main__.BoardState object at 0x1227e0f40>, 3: <__main__.BoardState object at 0x1227e0ee0>, 4: <__main__.BoardState object at 0x115778b80>, 5: <__main__.BoardState object at 0x1215f0a60>, 6: <__main__.BoardState object at 0x121a8bb20>, 7: <__main__.BoardState object at 0x121a88430>, 8: <__main__.BoardState object at 0x1227e34c0>, 9: <__main__.BoardState object at 0x1227e2a70>, 10: <__main__.BoardState object at 0x1227e3f70>, 11: <__main__.BoardState object at 0x1215f1db0>, 12: <__main__.BoardState object at 0x1227e3e20>, 13: <__main__.BoardState object at 0x1215f02b0>, 14: <__main__.BoardState object at 0x122224a60>, 15: <__main__.BoardState object at 0x122227ac0>, 16: <__main__.BoardState object at 0x115223f10>, 17: <__main__.BoardState object at 0x1152208e0>, 18: <__main__.BoardState object at 0x115d82fb0>, 19: <__main__.BoardState object at 0x1229a16c0>, 20: <__main__.BoardState obje

In [197]:
def minimax(board_st, plyer_tm, mx_depth):

    state_tree = generate_st_tree(board_st, plyr_team, mx_depth)
    q = []
    

In [198]:
def play_game(brd_state):

    brd_state.print_board()
    game_over = False
    while(game_over == False):
        val = input("move: start x, start y, end x, end y\n")
        tokens = val.split(",")
        tokens = [int(i) for i in tokens]
        
        brd_state.move_piece(tokens[1]-1, tokens[0]-1, tokens[3]-1, tokens[2]-1)
        print("Player 1 Moves: ")
        brd_state.print_board()
        print("")

        moves = brd_state.moves_available("Black")
        rand = randint(0,len(moves)-1)

        brd_state = moves[rand]
        print("Player 2 Moves: ")
        brd_state.print_board()
        print("")


In [199]:
def play_game(brd_state, string):

    brd_state.print_board()
    print("")
    num_moves = 0

    game_over = False
    
    while(game_over == False):

        moves = brd_state.moves_available("White")
        if len(moves) < 1:
            print("Player 2 wins in " + str(num_moves) + " moves")
            return
        rand = randint(0,len(moves)-1)

        brd_state = moves[rand]
        print("Player 1 Moves: ")
        brd_state.print_board()
        print("")
        num_moves += 1
        

        moves = brd_state.moves_available("Black")
        if len(moves) < 1:
            print("Player 1 wins in " + str(num_moves) + " moves")
            return
        rand = randint(0,len(moves)-1)

        brd_state = moves[rand]
        print("Player 2 Moves: ")
        brd_state.print_board()
        print("")
        num_moves += 1
        

In [200]:
play_game(new_game, "random moves")

x o x o x o x o 
o x o x o x o x 
x o x o x o x o 
o x o . o x o x 
x o x . x o x o 
o x o x o x o x 
x o x o x o x o 
o x o x o x o x 

Player 1 Moves: 
x o x o x o x o 
o x o x o x o x 
x o x o x o x o 
o x o . o x o x 
x o x o x o x o 
o x o . o x o x 
x o x . x o x o 
o x o x o x o x 

Player 2 Moves: 
x o x o x o x o 
o x o x o x o x 
x o x o x o x o 
o x o x . . o x 
x o x o x o x o 
o x o . o x o x 
x o x . x o x o 
o x o x o x o x 

Player 1 Moves: 
x o x o x o x o 
o x o x o x o x 
x o x o x o x o 
o x o x . . o x 
x o x o x o x o 
o x o . o x o x 
x o x o . . x o 
o x o x o x o x 

Player 2 Moves: 
x o x o x o x o 
o x o x o x o x 
x o x o x o x o 
o x o x . . o x 
x o x o x o x o 
o x o x o x o x 
x o x . . . x o 
o x o . o x o x 

Player 1 Moves: 
x o x o x o x o 
o x o x o x o x 
x o x o x o x o 
o x o x . . o x 
x o x . x o x o 
o x o . o x o x 
x o x o . . x o 
o x o . o x o x 

Player 2 Moves: 
x o x o x o x o 
o x o x o x o x 
x o x o x o x o 
o x o x . x o x 
x o x . 